In [72]:
!python precoss_dataset.py ../data/01_data_processed ./data.pkl

Graph(num_nodes={'user': 1970, 'wifi': 15336},
      num_edges={('user', 'relation', 'wifi'): 15405, ('wifi', 'relation-by', 'user'): 15405},
      metagraph=[('user', 'wifi', 'relation'), ('wifi', 'user', 'relation-by')])
Done!


Using backend: pytorch


In [73]:
!python ./model.py ./data.pkl --num-epochs 300 --num-workers 0 --device cpu --hidden-dims 64

{'train-graph': Graph(num_nodes={'user': 1970, 'wifi': 15336},
      num_edges={('user', 'relation', 'wifi'): 15405, ('wifi', 'relation-by', 'user'): 15405},
      metagraph=[('user', 'wifi', 'relation'), ('wifi', 'user', 'relation-by')]), 'train-indices': 1879      True
5509      True
34        True
11620     True
11178     True
         ...  
2581     False
12062    False
4732     False
10606    False
5566     False
Name: train_mask, Length: 1970, dtype: bool, 'val-indices': 1879     False
5509     False
34       False
11620    False
11178    False
         ...  
2581     False
12062    False
4732     False
10606    False
5566     False
Name: val_mask, Length: 1970, dtype: bool, 'test-indices': 1879     False
5509     False
34       False
11620    False
11178    False
         ...  
2581      True
12062     True
4732      True
10606     True
5566      True
Name: test_mask, Length: 1970, dtype: bool, 'item-texts': {}, 'item-images': None, 'user-type': 'user', 'item-type': 'wifi', 'use

Using backend: pytorch

100%|██████████| 5/5 [00:00<00:00, 20.63it/s]


In [ ]:
import dgl
??dgl.NID

In [13]:
import torch
torch.nn.Module

In [81]:
import numpy as np
import pandas as pd
import os

In [86]:
directory = '../data/01_data_processed'
users = pd.read_csv(os.path.join(directory,'user.csv'))
users = users[['customer_id','trans_time','target']]
users

,customer_id,trans_time,target
0,862201,2020-07-13 18:05:08,1
1,4271399,2020-07-27 10:25:27,1
2,4284127,2020-08-01 09:15:17,1
3,4237165,2020-07-03 13:05:09,0
4,4091456,2020-06-12 19:05:11,0
...,...,...,...
12102,4235548,2020-07-17 11:35:05,1
12103,4290699,2020-08-05 21:40:08,1
12104,4281305,2020-07-30 03:20:02,1
12105,4268429,2020-07-21 16:05:05,1


In [87]:
users.trans_time.min(),users.trans_time.max()

('2020-06-01 01:00:43', '2020-08-06 21:35:06')

In [88]:
users.target.value_counts().loc[0]/users.shape[0]

0.13603700338647062

In [91]:
wifi_list = pd.read_csv(os.path.join(directory,'wifi_list_id.csv'))
wifi_list = wifi_list[['customer_id','create_time','wifi_list']]
wifi_list = wifi_list.sort_values(['customer_id','wifi_list','create_time'])
wifi_list = wifi_list.drop_duplicates(['customer_id','wifi_list'])
wifi_list = wifi_list.rename(columns={'wifi_list':'wifi_id'})
wifi_list

,customer_id,create_time,wifi_id
25830,107056,2020-09-04 16:44:55,00:67:62:6d:02:e8
38719,107056,2020-09-08 19:46:41,2e:ff:ee:92:a8:03
38720,107056,2020-09-08 19:46:41,74:da:88:38:7f:2a
25826,107056,2020-09-04 16:44:55,80:26:89:58:d9:6b
25823,107056,2020-09-04 16:44:55,90:f6:52:ac:c3:c2
...,...,...,...
19207,4410632,2020-09-23 05:38:49,e8:65:d4:36:ec:30
19206,4410632,2020-09-23 05:38:49,f0:b4:d2:81:30:e6
19199,4410632,2020-09-23 05:38:49,f4:1d:6b:a5:88:70
19202,4410632,2020-09-23 05:38:49,fc:a6:cd:96:cd:f8


In [92]:
user_wifi_list = pd.merge(users,wifi_list,how='inner',on='customer_id')
user_wifi_list.shape

(15405, 5)

In [93]:
user_wifi_list[user_wifi_list['trans_time']>user_wifi_list['create_time']].shape

(15405, 5)

In [94]:
user_wifi_list

,customer_id,trans_time,target,create_time,wifi_id
0,4202015,2020-06-27 06:35:10,1,2020-06-24 16:22:09,0c:98:38:a9:f0:6d
1,4202015,2020-06-27 06:35:10,1,2020-06-24 16:22:09,0e:a8:a7:5d:ca:34
2,4202015,2020-06-27 06:35:10,1,2020-06-24 16:22:09,74:5b:c5:43:1f:9c
3,4202015,2020-06-27 06:35:10,1,2020-06-24 16:22:09,9a:c8:b8:90:5c:9b
4,4202015,2020-06-27 06:35:10,1,2020-06-24 16:22:09,f0:8c:fb:29:13:f0
...,...,...,...,...,...
15400,4235548,2020-07-17 11:35:05,1,2020-07-03 09:53:07,a0:ab:1b:70:ed:1c
15401,4235548,2020-07-17 11:35:05,1,2020-07-16 12:03:21,b4:e9:b0:9e:95:40
15402,4235548,2020-07-17 11:35:05,1,2020-07-16 12:03:21,b4:e9:b0:9e:95:4e
15403,4235548,2020-07-17 11:35:05,1,2020-07-16 12:03:21,b4:e9:b0:9e:95:4f


In [95]:
distinct_users_in_relation = wifi_list['customer_id'].unique()
distinct_wifi_in_relation = wifi_list['wifi_id'].unique()
users = users[users['customer_id'].isin(distinct_users_in_relation)]
wifi_list = wifi_list[wifi_list['wifi_id'].isin(distinct_wifi_in_relation)]

In [97]:
users.shape

(1970, 3)

In [98]:
users.target.value_counts()

1    1713
0     257
Name: target, dtype: int64

In [99]:
users.target.value_counts().loc[0]/users.shape[0]

0.1304568527918782

In [59]:
wifi_list.shape

(58262, 3)

In [70]:
wifi_list.drop_duplicates('wifi_id').shape

(57620, 3)

In [71]:
wifi_list

,customer_id,create_time,wifi_id
25830,107056,2020-09-04 16:44:55,00:67:62:6d:02:e8
38719,107056,2020-09-08 19:46:41,2e:ff:ee:92:a8:03
38720,107056,2020-09-08 19:46:41,74:da:88:38:7f:2a
25826,107056,2020-09-04 16:44:55,80:26:89:58:d9:6b
25823,107056,2020-09-04 16:44:55,90:f6:52:ac:c3:c2
...,...,...,...
19207,4410632,2020-09-23 05:38:49,e8:65:d4:36:ec:30
19206,4410632,2020-09-23 05:38:49,f0:b4:d2:81:30:e6
19199,4410632,2020-09-23 05:38:49,f4:1d:6b:a5:88:70
19202,4410632,2020-09-23 05:38:49,fc:a6:cd:96:cd:f8


In [61]:
entities = users['customer_id'].astype('category')
print(entities)
if not (entities.value_counts() == 1).all():
    raise ValueError('Different entity with the same primary key detected.')

6        4202015
16       4291583
22        880193
23       4148588
34       4220955
          ...   
12079    4121737
12084    4119054
12088    1315610
12091    4286858
12102    4235548
Name: customer_id, Length: 1970, dtype: category
Categories (1970, int64): [152917, 160937, 173091, 174326, ..., 4291897, 4291937, 4291980, 4291982]


In [69]:
entities = wifi_list['wifi_id'].astype('category')
print(entities)
if not (entities.value_counts() == 1).all():
    raise ValueError('Different entity with the same primary key detected.')

25830    00:67:62:6d:02:e8
38719    2e:ff:ee:92:a8:03
38720    74:da:88:38:7f:2a
25826    80:26:89:58:d9:6b
25823    90:f6:52:ac:c3:c2
               ...        
19207    e8:65:d4:36:ec:30
19206    f0:b4:d2:81:30:e6
19199    f4:1d:6b:a5:88:70
19202    fc:a6:cd:96:cd:f8
19198    fc:a6:cd:96:e5:e0
Name: wifi_id, Length: 58262, dtype: category
Categories (57620, object): [00:00:00:00:00:00, 00:01:23:01:4c:d0, 00:01:23:05:7e:08, 00:01:23:05:7f:a0, ..., fe:ec:da:f2:5b:f3, fe:ec:da:f2:5e:9f, fe:ec:da:f2:9f:17, fe:f9:d5:35:d9:5d]


ValueError: Different entity with the same primary key detected.

In [74]:
import dgl
import numpy as np
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.multiprocessing as mp
import dgl.nn.pytorch as dglnn
import time
import math
import argparse
from torch.nn.parallel import DistributedDataParallel
import tqdm

from utils import thread_wrapped_func
from load_graph import load_reddit, inductive_split

class SAGE(nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout):
        super().__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_classes = n_classes
        self.layers = nn.ModuleList()
        self.layers.append(dglnn.SAGEConv(in_feats, n_hidden, 'mean'))
        for i in range(1, n_layers - 1):
            self.layers.append(dglnn.SAGEConv(n_hidden, n_hidden, 'mean'))
        self.layers.append(dglnn.SAGEConv(n_hidden, n_classes, 'mean'))
        self.dropout = nn.Dropout(dropout)
        self.activation = activation

    def forward(self, blocks, x):
        h = x
        for l, (layer, block) in enumerate(zip(self.layers, blocks)):
            h = layer(block, h)
            if l != len(self.layers) - 1:
                h = self.activation(h)
                h = self.dropout(h)
        return h

    def inference(self, g, x, device):
        """
        Inference with the GraphSAGE model on full neighbors (i.e. without neighbor sampling).
        g : the entire graph.
        x : the input of entire node set.
        The inference code is written in a fashion that it could handle any number of nodes and
        layers.
        """
        # During inference with sampling, multi-layer blocks are very inefficient because
        # lots of computations in the first few layers are repeated.
        # Therefore, we compute the representation of all nodes layer by layer.  The nodes
        # on each layer are of course splitted in batches.
        # TODO: can we standardize this?
        for l, layer in enumerate(self.layers):
            y = th.zeros(g.num_nodes(), self.n_hidden if l != len(self.layers) - 1 else self.n_classes)

            sampler = dgl.dataloading.MultiLayerFullNeighborSampler(1)
            dataloader = dgl.dataloading.NodeDataLoader(
                g,
                th.arange(g.num_nodes()),
                sampler,
                batch_size=args.batch_size,
                shuffle=True,
                drop_last=False,
                num_workers=args.num_workers)

            for input_nodes, output_nodes, blocks in tqdm.tqdm(dataloader):
                block = blocks[0]

                block = block.int().to(device)
                h = x[input_nodes].to(device)
                h = layer(block, h)
                if l != len(self.layers) - 1:
                    h = self.activation(h)
                    h = self.dropout(h)

                y[output_nodes] = h.cpu()

            x = y
        return y

def compute_acc(pred, labels):
    """
    Compute the accuracy of prediction given the labels.
    """
    return (th.argmax(pred, dim=1) == labels).float().sum() / len(pred)

def evaluate(model, g, nfeat, labels, val_nid, device):
    """
    Evaluate the model on the validation set specified by ``val_nid``.
    g : The entire graph.
    inputs : The features of all the nodes.
    labels : The labels of all the nodes.
    val_nid : A node ID tensor indicating which nodes do we actually compute the accuracy for.
    device : The GPU device to evaluate on.
    """
    model.eval()
    with th.no_grad():
        pred = model.inference(g, nfeat, device)
    model.train()
    return compute_acc(pred[val_nid], labels[val_nid])

def load_subtensor(nfeat, labels, seeds, input_nodes, dev_id):
    """
    Extracts features and labels for a subset of nodes.
    """
    batch_inputs = nfeat[input_nodes].to(dev_id)
    batch_labels = labels[seeds].to(dev_id)
    return batch_inputs, batch_labels

#### Entry point

def run(proc_id, n_gpus, args, devices, data):
    # Start up distributed training, if enabled.
    dev_id = devices[proc_id]
    if n_gpus > 1:
        dist_init_method = 'tcp://{master_ip}:{master_port}'.format(
            master_ip='127.0.0.1', master_port='12345')
        world_size = n_gpus
        th.distributed.init_process_group(backend="nccl",
                                          init_method=dist_init_method,
                                          world_size=world_size,
                                          rank=proc_id)
    th.cuda.set_device(dev_id)

    # Unpack data
    n_classes, train_g, val_g, test_g = data

    if args.inductive:
        train_nfeat = train_g.ndata.pop('features')
        val_nfeat = val_g.ndata.pop('features')
        test_nfeat = test_g.ndata.pop('features')
        train_labels = train_g.ndata.pop('labels')
        val_labels = val_g.ndata.pop('labels')
        test_labels = test_g.ndata.pop('labels')
    else:
        train_nfeat = val_nfeat = test_nfeat = g.ndata.pop('features')
        train_labels = val_labels = test_labels = g.ndata.pop('labels')

    if not args.data_cpu:
        train_nfeat = train_nfeat.to(dev_id)
        train_labels = train_labels.to(dev_id)

    in_feats = train_nfeat.shape[1]

    train_mask = train_g.ndata['train_mask']
    val_mask = val_g.ndata['val_mask']
    test_mask = ~(test_g.ndata['train_mask'] | test_g.ndata['val_mask'])
    train_nid = train_mask.nonzero().squeeze()
    val_nid = val_mask.nonzero().squeeze()
    test_nid = test_mask.nonzero().squeeze()

    # Split train_nid
    train_nid = th.split(train_nid, math.ceil(len(train_nid) / n_gpus))[proc_id]

    # Create PyTorch DataLoader for constructing blocks
    sampler = dgl.dataloading.MultiLayerNeighborSampler(
        [int(fanout) for fanout in args.fan_out.split(',')])
    dataloader = dgl.dataloading.NodeDataLoader(
        train_g,
        train_nid,
        sampler,
        batch_size=args.batch_size,
        shuffle=True,
        drop_last=False,
        num_workers=args.num_workers)

    # Define model and optimizer
    model = SAGE(in_feats, args.num_hidden, n_classes, args.num_layers, F.relu, args.dropout)
    model = model.to(dev_id)
    if n_gpus > 1:
        model = DistributedDataParallel(model, device_ids=[dev_id], output_device=dev_id)
    loss_fcn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    # Training loop
    avg = 0
    iter_tput = []
    for epoch in range(args.num_epochs):
        tic = time.time()

        # Loop over the dataloader to sample the computation dependency graph as a list of
        # blocks.
        for step, (input_nodes, seeds, blocks) in enumerate(dataloader):
            if proc_id == 0:
                tic_step = time.time()

            # Load the input features as well as output labels
            batch_inputs, batch_labels = load_subtensor(train_nfeat, train_labels,
                                                        seeds, input_nodes, dev_id)
            blocks = [block.int().to(dev_id) for block in blocks]
            # Compute loss and prediction
            batch_pred = model(blocks, batch_inputs)
            loss = loss_fcn(batch_pred, batch_labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if proc_id == 0:
                iter_tput.append(len(seeds) * n_gpus / (time.time() - tic_step))
            if step % args.log_every == 0 and proc_id == 0:
                acc = compute_acc(batch_pred, batch_labels)
                print('Epoch {:05d} | Step {:05d} | Loss {:.4f} | Train Acc {:.4f} | Speed (samples/sec) {:.4f} | GPU {:.1f} MB'.format(
                    epoch, step, loss.item(), acc.item(), np.mean(iter_tput[3:]), th.cuda.max_memory_allocated() / 1000000))

        if n_gpus > 1:
            th.distributed.barrier()

        toc = time.time()
        if proc_id == 0:
            print('Epoch Time(s): {:.4f}'.format(toc - tic))
            if epoch >= 5:
                avg += toc - tic
            if epoch % args.eval_every == 0 and epoch != 0:
                if n_gpus == 1:
                    eval_acc = evaluate(
                        model, val_g, val_nfeat, val_labels, val_nid, devices[0])
                    test_acc = evaluate(
                        model, test_g, test_nfeat, test_labels, test_nid, devices[0])
                else:
                    eval_acc = evaluate(
                        model.module, val_g, val_nfeat, val_labels, val_nid, devices[0])
                    test_acc = evaluate(
                        model.module, test_g, test_nfeat, test_labels, test_nid, devices[0])
                print('Eval Acc {:.4f}'.format(eval_acc))
                print('Test Acc: {:.4f}'.format(test_acc))


    if n_gpus > 1:
        th.distributed.barrier()
    if proc_id == 0:
        print('Avg epoch time: {}'.format(avg / (epoch - 4)))

if __name__ == '__main__':
    argparser = argparse.ArgumentParser("multi-gpu training")
    argparser.add_argument('--gpu', type=str, default='0',
                           help="Comma separated list of GPU device IDs.")
    argparser.add_argument('--num-epochs', type=int, default=20)
    argparser.add_argument('--num-hidden', type=int, default=16)
    argparser.add_argument('--num-layers', type=int, default=2)
    argparser.add_argument('--fan-out', type=str, default='10,25')
    argparser.add_argument('--batch-size', type=int, default=1000)
    argparser.add_argument('--log-every', type=int, default=20)
    argparser.add_argument('--eval-every', type=int, default=5)
    argparser.add_argument('--lr', type=float, default=0.003)
    argparser.add_argument('--dropout', type=float, default=0.5)
    argparser.add_argument('--num-workers', type=int, default=0,
                           help="Number of sampling processes. Use 0 for no extra process.")
    argparser.add_argument('--inductive', action='store_true',
                           help="Inductive learning setting")
    argparser.add_argument('--data-cpu', action='store_true',
                           help="By default the script puts all node features and labels "
                                "on GPU when using it to save time for data copy. This may "
                                "be undesired if they cannot fit in GPU memory at once. "
                                "This flag disables that.")
    args = argparser.parse_args()
    
    devices = list(map(int, args.gpu.split(',')))
    n_gpus = len(devices)

    g, n_classes = load_reddit()
    # Construct graph
    g = dgl.as_heterograph(g)

    if args.inductive:
        train_g, val_g, test_g = inductive_split(g)
    else:
        train_g = val_g = test_g = g

    # Create csr/coo/csc formats before launching training processes with multi-gpu.
    # This avoids creating certain formats in each sub-process, which saves momory and CPU.
    train_g.create_formats_()
    val_g.create_formats_()
    test_g.create_formats_()
    # Pack data
    data = n_classes, train_g, val_g, test_g

    if n_gpus == 1:
        run(0, n_gpus, args, devices, data)
    else:
        procs = []
        for proc_id in range(n_gpus):
            p = mp.Process(target=thread_wrapped_func(run),
                           args=(proc_id, n_gpus, args, devices, data))
            p.start()
            procs.append(p)
        for p in procs:
            p.join()


ModuleNotFoundError: No module named 'utils'